In [1]:
import pandas as pd

qualified_movies_df = pd.read_csv('.\\data\\processed_movies\\qualified_movies.csv')
qualified_movies_df = qualified_movies_df.fillna('')

## Create text processing object

In [2]:
from utils.text_processing import TextProcessing

text_processing_obj = TextProcessing()

## Add keywords from overview into the keywords column, clean punct and merge them in new column

In [3]:
qualified_movies_df['list_of_keywords_overview'] = qualified_movies_df['overview']. \
    apply(lambda x: text_processing_obj.extract_keywords_overview(x))

qualified_movies_df['list_of_keywords_overview'] = qualified_movies_df. \
    apply(lambda x: text_processing_obj.clear_keywords_overview(x.overview, x.list_of_keywords_overview), axis=1)

qualified_movies_df['merged_keywords'] = \
    qualified_movies_df.apply(lambda x: text_processing_obj.merge_overview_keywords_and_keywords
    (x.list_of_keywords_overview, x.keywords), axis=1)

In [4]:
"""Save the dataframe here"""

qualified_movies_df[['movie_id', 'overview', 'keywords', 'list_of_keywords_overview', 'merged_keywords']]. \
    to_csv('.\\data\\nlp\\merged_keywords.csv')

## Load merged keywords CSV and continue processing

In [5]:
import pandas as pd
"""Every time we load the df we are filling Nans with empty string"""
qualified_movies_df = pd.read_csv('.\\data\\nlp\\merged_keywords.csv')
qualified_movies_df = qualified_movies_df.fillna('')

In [6]:
from utils.text_processing import TextProcessing

"""redefine class object"""
text_processing_obj = TextProcessing()

In [7]:
dict_of_root_keywords = text_processing_obj.create_dict_of_root_keywords(qualified_movies_df, column='merged_keywords')

In [8]:
"""check the dictionary of root keywords"""
dict_of_root_keywords

{'start': {'start', 'started', 'starting', 'starts'},
 'independent film': {'independent film'},
 'thing': {'thing', 'things'},
 'jail': {'jail', 'jailed', 'jails'},
 'escap': {'escape', 'escaped', 'escapes', 'escaping'},
 'crime': {'crime', 'crimes'},
 'dream': {'dream', 'dreamed', 'dreaming', 'dreams'},
 'falling in lov': {'falling in love'},
 'underdog': {'underdog'},
 'taisto kasurinen': {'taisto kasurinen'},
 'factory work': {'factory worker'},
 'helsinki': {'helsinki'},
 'committed suicid': {'committed suicide'},
 'prison': {'prison', 'prisoner', 'prisoners', 'prisons'},
 'planned ...': {'planned ...'},
 'life': {'life'},
 'frame': {'frame', 'framed', 'frames'},
 'countri': {'countries', 'country'},
 'leav': {'leave', 'leaves', 'leaving'},
 'finnish coal min': {'finnish coal miner'},
 'commit': {'commit',
  'commitment',
  'commitments',
  'commits',
  'committed',
  'committing'},
 'father': {'father', 'fathered', 'fathering', 'fathers'},
 'bet': {'bet', 'bets', 'betting'},
 'un

In [9]:
print("Number of unique keywords: {}".format(len(dict_of_root_keywords.keys())))

"""Example of words that have the same root but different as keywords in our dataset"""
for index, root_key in enumerate(dict_of_root_keywords.keys()):
    if len(dict_of_root_keywords[root_key]) > 1:
        print(dict_of_root_keywords[root_key], len(dict_of_root_keywords[root_key]))
    if index == 50:
        break

Number of unique keywords: 69923
{'starts', 'started', 'starting', 'start'} 4
{'thing', 'things'} 2
{'jailed', 'jails', 'jail'} 3
{'escaped', 'escape', 'escapes', 'escaping'} 4
{'crime', 'crimes'} 2
{'dreaming', 'dreamed', 'dreams', 'dream'} 4
{'prisoners', 'prison', 'prisoner', 'prisons'} 4
{'framed', 'frames', 'frame'} 3
{'country', 'countries'} 2
{'leave', 'leaving', 'leaves'} 3
{'committing', 'committed', 'commit', 'commits', 'commitment', 'commitments'} 6
{'fathers', 'fathered', 'fathering', 'father'} 4
{'bet', 'bets', 'betting'} 3
{'witch', 'witches'} 2
{'serving', 'serves', 'serve', 'served'} 4
{'night', 'nights'} 2
{'evening', 'evenings'} 2
{'placing', 'place', 'places'} 3
{'hoodlums', 'hoodlum'} 2
{'hotel room', 'hotel rooms'} 2
{'outrageous predicament', 'outrageous predicaments'} 2
{'raced', 'racing', 'race', 'races'} 4
{'paths', 'path'} 2


In [10]:
"""Thats why we will normalize the keywords by using only 1 keyword per root"""


dict_of_root_selected_keyword = text_processing_obj.choose_best_keyword_for_root(dict_of_root_keywords)

In [13]:
dict_of_root_selected_keyword

{'crime': 'crime',
 'leav': 'leave',
 'underdog': 'underdog',
 'prison': 'prison',
 'finnish coal min': 'finnish coal miner',
 'life': 'life',
 'start': 'start',
 'thing': 'thing',
 'escap': 'escape',
 'dream': 'dream',
 'commit': 'commit',
 'jail': 'jail',
 'countri': 'country',
 'committed suicid': 'committed suicide',
 'helsinki': 'helsinki',
 'falling in lov': 'falling in love',
 'father': 'father',
 'factory work': 'factory worker',
 'frame': 'frame',
 'independent film': 'independent film',
 'planned ...': 'planned ...',
 'hotel': 'hotel',
 'los angel': 'los angeles',
 'bet': 'bet',
 'episode film': 'episode film',
 'night': 'night',
 'unusual guest': 'unusual guests',
 'place': 'place',
 'even': 'evening',
 'outrageous predica': 'outrageous predicament',
 'hotel room': 'hotel room',
 'woman director': 'woman director',
 'hoodlum': 'hoodlum',
 'job ...': 'job ...',
 'witch': 'witch',
 'sperm': 'sperm',
 'ted': 'ted',
 'serv': 'serve',
 "new year's ev": "new year's eve",
 'room se

In [11]:
"""Normalize keywords, we use this to replace all keywords with the selected keywords from the root dict"""

qualified_movies_df['normalized_keywords'] = \
    qualified_movies_df.apply(lambda x: text_processing_obj.normalize_keywords(
        x.merged_keywords, dict_of_root_selected_keyword), axis=1)

No key Γη in dict


In [12]:
qualified_movies_df[
    ['movie_id', 'overview', 'keywords', 'list_of_keywords_overview', 'merged_keywords', 'normalized_keywords']]. \
    to_csv('.\\data\\nlp\\normalized_keywords.csv')

## Do the final processing on the whole dataset, including synonyms replacement

In [10]:
import pandas as pd
"""Every time we load the df we are filling Nans with empty string"""
qualified_movies_df = pd.read_csv('.\\data\\nlp\\normalized_keywords.csv')
qualified_movies_df = qualified_movies_df.fillna('')

In [11]:
from utils.text_processing import TextProcessing

"""redefine class object"""
text_processing_obj = TextProcessing()

In [12]:
"""whats the minimum count for a word to be replaced by synonym or deleted"""
count_threshold = 2

qualified_movies_df['normalized_keywords'] = qualified_movies_df['normalized_keywords']. \
    apply(lambda x: text_processing_obj.remove_stopwords(x))

dict_of_keyword_count = text_processing_obj.count_words_occurences(qualified_movies_df, 'normalized_keywords')

In [13]:
qualified_movies_df['processed_keywords'] = \
    qualified_movies_df.apply(lambda x: text_processing_obj.process_n_grams(dict_of_keyword_count=dict_of_keyword_count,
                                                                            count_threshold=count_threshold,
                                                                            list_of_keywords=x.normalized_keywords), axis=1)

qualified_movies_df['processed_keywords'] = \
    qualified_movies_df.apply(
        lambda x: text_processing_obj.process_keywords_with_synonyms(dict_of_keyword_count=dict_of_keyword_count,
                                                                     count_threshold=count_threshold,
                                                                     list_of_keywords=x.processed_keywords), axis=1)

qualified_movies_df['processed_keywords'] = \
    qualified_movies_df.apply(
        lambda x: text_processing_obj.final_clean_special_symbols(list_of_keywords=x.processed_keywords), axis=1)

"""clear words that appear more than one time"""
qualified_movies_df['processed_keywords'] = \
    qualified_movies_df['processed_keywords'].apply(lambda x: list(set(x)))


qualified_movies_df['preprocessed_keywords'] = \
    qualified_movies_df['processed_keywords'].apply(lambda x: text_processing_obj.format_keywords(x))

dory --> rowboat : 4 count
bench --> judiciary : 3 count
hugging --> fondling : 5 count
magnate --> power : 189 count
saviour --> jesus christ : 10 count
pillage --> loot : 16 count
payback --> vengeance : 40 count
gulf --> disconnect : 3 count
semen --> seed : 6 count
epa --> environmental protection agency : 3 count
dome --> attic : 19 count
fairytale --> fairy tale : 45 count
lowlife --> rat : 18 count
tweed --> white : 22 count
swinger --> tramp : 9 count
houseman --> intern : 13 count
hotshot --> star : 100 count
central --> exchange : 25 count
guerillas --> insurgent : 14 count
overlap --> lap : 3 count
cartel --> trust : 63 count
mortification --> chagrin : 9 count
sum --> join : 149 count
digger --> shovel : 4 count
password --> word : 27 count
lunatic --> maniac : 23 count
excommunication --> exclusion : 4 count
platform --> program : 18 count
platform --> program : 18 count
railway --> railroad : 14 count
platform --> program : 18 count
bloom --> flower : 16 count
nip --> sho


adam --> ecstasy : 13 count
prep --> homework : 4 count
austen --> jane austen : 9 count
bloom --> flower : 16 count
heyday --> flower : 16 count
victor --> master : 57 count
outrage --> scandal : 19 count
wow --> scream : 19 count
ace --> star : 100 count
rapture --> transport : 51 count
jesus --> jesus christ : 10 count
setbacks --> blow : 24 count
chorine --> chorus girl : 3 count
victor --> master : 57 count
rapier --> tuck : 3 count
lark --> escapade : 12 count
shortcomings --> defect : 4 count
enmity --> antagonism : 3 count
bride-to-be --> fiancee : 10 count
sequence --> episode : 12 count
bob --> dock : 5 count
central --> exchange : 25 count
grand --> yard : 7 count
cattleman --> cowboy : 32 count
climax --> coming : 61 count
gangland --> organized crime : 39 count
spree --> fling : 8 count
collar --> arrest : 71 count
goings --> release : 112 count
porter --> doorman : 3 count
wardrobe --> press : 20 count
workshop --> shop : 37 count
bloom --> flower : 16 count
scruples -->


central --> exchange : 25 count
wrench --> pull : 72 count
oxen --> cattle : 12 count
beacon --> lighthouse : 9 count
footprint --> step : 42 count
domain --> land : 146 count
deuce --> devil : 36 count
madcap --> swashbuckler : 10 count
booby --> pinhead : 9 count
norm --> average : 3 count
helper --> benefactor : 3 count
lame --> square : 8 count
grand --> yard : 7 count
menu --> card : 5 count
brat --> terror : 165 count
statement --> command : 50 count
roach --> circle : 23 count
avatar --> incarnation : 3 count
messenger --> courier : 3 count
assess --> bottom : 18 count
wagon --> station wagon : 4 count
overlap --> lap : 3 count
swinger --> tramp : 9 count
bob --> dock : 5 count
exemplar --> model : 25 count
spike --> ear : 12 count
hex --> curse : 52 count
sewerage --> sewer : 16 count
hockey --> ice hockey : 13 count
clergyman --> reverend : 5 count
avant-garde --> van : 22 count
soil --> land : 146 count
debate --> argument : 11 count
dick --> tool : 9 count
tone --> spirit :


stool --> throne : 34 count
variation --> sport : 222 count
frolic --> play : 151 count
caribou --> reindeer : 4 count
dubya --> george w. bush : 4 count
conundrum --> riddle : 28 count
monstrosity --> monster : 150 count
outfit --> rig : 4 count
mare --> maria : 3 count
passageway --> passage : 4 count
fad --> cult : 39 count
beacon --> lighthouse : 9 count
sleeve --> arm : 65 count
eighties --> 80 : 7 count
schoolmate --> classmate : 30 count
slumber --> sleep : 36 count
missteps --> trip : 75 count
tune --> air : 24 count
backbone --> sand : 9 count
roller --> roll : 14 count
portions --> fate : 105 count
veterinary --> veterinarian : 8 count
stiff --> corpse : 42 count
vacuum --> void : 3 count
bowels --> gut : 5 count
wipeout --> demolition : 3 count
cap --> capital : 27 count
enchantress --> witch : 89 count
steward --> flight attendant : 6 count
font --> face : 274 count
typeface --> face : 274 count
blanket --> cover : 45 count
watcher --> security guard : 25 count
adam --> ec


jolt --> shock : 53 count
instant --> moment : 38 count
outfit --> rig : 4 count
belligerent --> fighter : 24 count
simpleton --> simple : 26 count
whiplash --> whip : 12 count
sleeve --> arm : 65 count
wiener --> dog : 146 count
wagon --> station wagon : 4 count
daytime --> day : 465 count
dixie --> south : 32 count
victor --> master : 57 count
tablet --> pill : 11 count
tablet --> pill : 11 count
usher --> guide : 22 count
ouija --> ouija board : 8 count
lunatic --> maniac : 23 count
sheet --> plane : 32 count
dorm --> dormitory : 7 count
stiff --> corpse : 42 count
grandchildren --> grandchild : 3 count
shreds --> rag : 15 count
trough --> bowl : 8 count
fortuneteller --> fortune teller : 12 count
orient --> east : 15 count
orb --> globe : 24 count
orb --> globe : 24 count
storekeeper --> shopkeeper : 3 count
instant --> moment : 38 count
instant --> moment : 38 count
pup --> puppy : 13 count
scamp --> monkey : 29 count
lance --> gig : 6 count
amour --> affair : 93 count
ringer -->


casualty --> fatal accident : 3 count
despatch --> dispatch : 26 count
hitler --> adolf hitler : 14 count
automobile --> car : 107 count
guise --> pretense : 3 count
hotshot --> star : 100 count
maiden --> maid : 24 count
stockade --> concentration camp : 14 count
djinn --> genie : 4 count
email --> e-mail : 3 count
swinger --> tramp : 9 count
ace --> star : 100 count
jailbreak --> break : 121 count
orient --> east : 15 count
dick --> tool : 9 count
illusionist --> magician : 10 count
peddler --> drug trafficker : 9 count
congregation --> fold : 4 count
quintet --> 5 : 13 count
heartbeats --> beat : 45 count
usher --> guide : 22 count
dough --> loot : 16 count
trustee --> legal guardian : 3 count
tradesman --> shopkeeper : 3 count
gentlemen --> valet : 7 count
gentlemen --> valet : 7 count
goat --> butt : 4 count
madcap --> swashbuckler : 10 count
coke --> snow : 47 count
grey --> gray : 3 count
bowman --> archer : 5 count
adam --> ecstasy : 13 count
counterpart --> twin : 31 count
sk


shooter --> gun : 95 count
foreboding --> premonition : 10 count
exposure --> photograph : 71 count
veterinary --> veterinarian : 8 count
wrench --> pull : 72 count
renaissance --> rebirth : 6 count
labor --> task : 56 count
oss --> bone : 16 count
grandchildren --> grandchild : 3 count
favour --> favor : 14 count
jester --> fool : 11 count
gem --> treasure : 44 count
debate --> argument : 11 count
goat --> butt : 4 count
artillery --> gun : 95 count
wiles --> shenanigan : 5 count
booty --> loot : 16 count
trader --> dealer : 9 count
metro --> subway : 28 count
booby --> pinhead : 9 count
handler --> coach : 31 count
stooge --> butt : 4 count
epitome --> image : 23 count
parson --> minister : 19 count
seafarer --> jack : 21 count
gunpoint --> point : 40 count
ft --> foot : 15 count
womanhood --> woman : 281 count
capacity --> content : 7 count
huntsman --> hunter : 28 count
throwers --> potter : 3 count
behaviour --> behavior : 13 count
trader --> dealer : 9 count
payback --> vengeanc


hollow --> hole : 18 count
menu --> card : 5 count
helper --> benefactor : 3 count
livelihood --> support : 47 count
overflow --> flood : 23 count
foreboding --> premonition : 10 count
avail --> service : 27 count
nightlife --> night life : 13 count
bloom --> flower : 16 count
rental --> lease : 8 count
gilbert --> gi : 4 count
adam --> ecstasy : 13 count
tablet --> pill : 11 count
brat --> terror : 165 count
labor --> task : 56 count
sasquatch --> bigfoot : 4 count
militant --> activist : 9 count
dear --> love : 907 count
vagrant --> drifter : 17 count
distortion --> torture : 117 count
summertime --> summer : 118 count
freshmen --> newcomer : 10 count
relief --> rest : 85 count
policewomen --> policewoman : 5 count
excursion --> expedition : 26 count
dusk --> fall : 362 count
virtuoso --> star : 100 count
frenzy --> fury : 6 count
cracker --> redneck : 10 count
rhythm --> beat : 45 count
stern --> bottom : 18 count
analysis --> psychoanalysis : 8 count
exposure --> photograph : 71 c

sneakers --> sneak : 9 count
camouflages --> disguise : 39 count
vet --> veteran : 11 count
vet --> veteran : 11 count
soil --> land : 146 count
consultant --> advisor : 4 count
context --> circumstance : 15 count
labor --> task : 56 count
scourge --> terror : 165 count
confederacy --> conspiracy : 85 count
debate --> argument : 11 count
myanmar --> burma : 3 count
permission --> license : 4 count
statement --> command : 50 count
shooter --> gun : 95 count
shooter --> gun : 95 count
banner --> standard : 4 count
fishermen --> fisherman : 12 count
pseudo --> fraud : 20 count
porter --> doorman : 3 count
fringes --> outskirt : 9 count
adam --> ecstasy : 13 count
pm --> prime minister : 13 count
division --> class : 49 count
motherland --> homeland : 24 count
division --> class : 49 count
sheet --> plane : 32 count
roma --> rome : 51 count
debate --> argument : 11 count
division --> class : 49 count
houseman --> intern : 13 count
email --> e-mail : 3 count
grandchildren --> grandchild : 3

In [14]:
qualified_movies_df[['movie_id', 'preprocessed_keywords']].to_csv(
    '.\\data\\nlp\\preprocessed_keywords.csv', index=False)

In [15]:
keywords = qualified_movies_df['processed_keywords']

In [16]:
qualified_movies_df['processed_keywords']

0        [country, jail, frame, helsinki, escape, indep...
1        [evening, ted, hotel, hotel room, place, episo...
2        [bargain, henchman, savage, vicious, directly,...
3        [galaxy, evil, stormtrooper, totalitarian, eff...
4        [hungry, loss, underwater, hypnotic, rowboat, ...
                               ...                        
11392    [hide, safe, travel, left, identity, fugitive,...
11393    [woman, uncover, retrace, suspect, memory, pla...
11394    [team, area, testify, hijack, trial, desperate...
11395    [prison, team, recover, attempt, ends, scorpio...
11396    [face, adversary, find, asks, rich, thief, aut...
Name: processed_keywords, Length: 11397, dtype: object

In [17]:
count_keywords = dict()

for list_of_keywords in keywords:
    for keyword in list_of_keywords:
        if keyword in count_keywords.keys():
            count_keywords[keyword] += 1
        else:
            count_keywords[keyword] = 1

In [18]:
sorted(count_keywords.items(), key=lambda x: x[1], reverse=True)

[('find', 1522),
 ('life', 1480),
 ('love', 1091),
 ('live', 1074),
 ('murder', 1021),
 ('family', 986),
 ('friend', 975),
 ('young', 707),
 ('woman director', 703),
 ('day', 659),
 ('work', 637),
 ('father', 628),
 ('turn', 618),
 ('woman', 616),
 ('death', 572),
 ('independent film', 562),
 ('fall', 543),
 ('kill', 538),
 ('wife', 537),
 ('discover', 535),
 ('fight', 528),
 ('girl', 525),
 ('war', 512),
 ('group', 503),
 ('daughter', 490),
 ('mother', 478),
 ('force', 474),
 ('secret', 473),
 ('return', 468),
 ('meet', 466),
 ('son', 460),
 ('police', 452),
 ('save', 438),
 ('revenge', 436),
 ('power', 431),
 ('sex', 423),
 ('based on novel', 418),
 ('violence', 415),
 ('town', 410),
 ('plan', 408),
 ('team', 408),
 ('music', 405),
 ('escape', 403),
 ('school', 401),
 ('face', 400),
 ('people', 396),
 ('relationship', 395),
 ('dream', 389),
 ('duringcreditsstinger', 387),
 ('lead', 383),
 ('learn', 380),
 ('prison', 377),
 ('follow', 375),
 ('start', 374),
 ('city', 372),
 ('friendsh

## Word Cloud

In [9]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [10]:
wc = WordCloud(width=800, height=400)

In [11]:
wc.fit_words(count_keywords)
wc.to_file('output1.png')